In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score
import warnings


In [2]:
def f1_score_eval(preds, labels):
    preds = preds.values.reshape(-1,)
    labels = labels.values.reshape(-1,)
    scores = f1_score(y_true=labels, y_pred=preds, labels=[0,1,2], average=None)
    scores = scores[0] * 0.2 + scores[1] * 0.2 + scores[2] * 0.6
    return scores


In [3]:
if __name__ == "__main__":

    train_select = {'20190701': 5261, '20190702': 5354, '20190703': 5173, '20190704': 5178, '20190705': 4818,
                    '20190706': 4818, '20190707': 5128, '20190708': 5150, '20190709': 5001, '20190710': 5302,
                    '20190711': 5514, '20190712': 5454, '20190713': 5307, '20190714': 5054, '20190715': 5480,
                    '20190716': 5436, '20190717': 4998, '20190718': 5087, '20190719': 4951, '20190720': 4965,
                    '20190721': 5284, '20190722': 5346, '20190723': 5102, '20190724': 5075, '20190725': 5367}
    for tk, tv in train_select.items():
        train_select[tk] = tv

    valid_select = {'20190726': 5241, '20190727': 4876, '20190728': 5260, '20190729': 5367, '20190730': 5344}
    

In [4]:
    #m1_valid = pd.read_csv('valid_pred_xgb.csv', index_col=0, nrows=26088)
    #m1_valid['label'] = m1_valid['label'] + 1
    #m1_valid.to_csv('valid_pred_xgb.csv', index=False, encoding='utf8')
    #m1_valid = pd.read_csv('test_pred_DNN.csv')
    #m1_valid['label'] = m1_valid['label'] + 1
    #m1_valid.to_csv('test_pred_DNN.csv', index=False, encoding='utf8')

In [5]:
    # [0.60,0.45] 两个lgb 0.5557662040556728
    # [0.60,0.31] lgb,NN 0.5601825134790559
    is_valid = True  # 融合用于验证还是测试，True为验证
    is_soft = False
    m0 = '_set_pred_www.csv'
    m1 = '_pred_DNN2_sim_0.561.csv'
    m2 = '_result-lr-5509804.csv'
    ensemble_weight = [0.45,0.01,0.45]  # 两个模型的权重，分数更高的权重也要更高，无需和为1
    m0_valid = pd.read_csv(('valid' if is_valid is True else 'test')+m0)  # 模型1输出文件
    m1_valid = pd.read_csv(('valid' if is_valid is True else 'test')+m1)  # 模型2输出文件
    m2_valid = pd.read_csv(('valid' if is_valid is True else 'test')+m2)  # 模型2输出文件
    

In [6]:
    valid = pd.DataFrame()
    for valid_day, n_row in valid_select.items():
        valid_tmp = pd.read_csv(f"../feature/is_train_{valid_day}.csv", usecols=['label'], dtype=int)
        valid_tmp['id'] = range(0, valid_tmp.shape[0])
        valid_row = pd.read_csv(f"../adversarial_validation/adversarial_validation_{valid_day}.csv", nrows=n_row)
        valid_tmp = valid_tmp.merge(valid_row, on='id', how='right')
        valid_tmp.drop(['id', 'preds'], axis=1, inplace=True)
        valid = pd.concat([valid, valid_tmp], axis=0, ignore_index=True)
    del valid_tmp, valid_row

In [7]:
    # 三个模型权重测试
    if is_valid == True:
        ensemble_weight = [0.45,0,0]  # 两个模型的权重，分数更高的权重也要更高，无需和为1
        m0_valid = pd.read_csv(('valid' if is_valid is True else 'test')+m0)  # 模型1输出文件
        m1_valid = pd.read_csv(('valid' if is_valid is True else 'test')+m1)  # 模型2输出文件
        m2_valid = pd.read_csv(('valid' if is_valid is True else 'test')+m2)  # 模型2输出文件
        ran = list(np.arange(0.0,0.6,0.01))
        best_s_score = 0
        best_h_score = 0
        for w in tqdm(ran):
            for w2 in ran:

                ensemble_valid = pd.DataFrame()
                ensemble_valid['model_0_stat'] = m0_valid['label'].astype(int) - 1
                ensemble_valid['model_1_stat'] = m1_valid['label'].astype(int) - 1
                ensemble_valid['model_2_stat'] = m2_valid['label'].astype(int) - 1


                for stat in range(1, 4):
                    ensemble_valid[f'{stat}_pred_prob'] = ensemble_weight[0] * m0_valid.loc[:, f'{stat}_pred_prob'] + \
                                                          w * m1_valid.loc[:, f'{stat}_pred_prob'] + \
                                                          w2 * m2_valid.loc[:, f'{stat}_pred_prob']
                    m0_valid[f'{stat}_pred_label'] = 0
                    m0_valid.loc[m0_valid[m0_valid.label == stat].index.tolist(), f'{stat}_pred_label'] = 1
                    m1_valid[f'{stat}_pred_label'] = 0
                    m1_valid.loc[m1_valid[m1_valid.label == stat].index.tolist(), f'{stat}_pred_label'] = 1
                    m2_valid[f'{stat}_pred_label'] = 0
                    m2_valid.loc[m2_valid[m2_valid.label == stat].index.tolist(), f'{stat}_pred_label'] = 1

                    ensemble_valid[f'{stat}_pred_label'] = ensemble_weight[0] * m0_valid.loc[:, f'{stat}_pred_label'] + \
                                                   w * m1_valid.loc[:, f'{stat}_pred_label'] + \
                                                   w2 * m2_valid.loc[:, f'{stat}_pred_label']

                ensemble_valid['pred_stat_soft'] = ensemble_valid.loc[:, [f'{stat}_pred_prob' for stat in range(1, 4)]].idxmax(axis=1)
                ensemble_valid['pred_stat_soft'] = ensemble_valid['pred_stat_soft'].apply(lambda x: int(x.split('_')[0]) - 1).astype('int')
                ensemble_valid['pred_stat_hard'] = ensemble_valid.loc[:, [f'{stat}_pred_label' for stat in range(1, 4)]].idxmax(axis=1)
                ensemble_valid['pred_stat_hard'] = ensemble_valid['pred_stat_hard'].apply(lambda x: int(x.split('_')[0]) - 1).astype('int')

                curr_s_score = f1_score_eval(ensemble_valid["pred_stat_soft"], valid["label"])
                curr_h_score = f1_score_eval(ensemble_valid["pred_stat_hard"], valid["label"])
                #print(f'weight: {w},{w2}, soft ensemble valid F1 score: {curr_score}')
                if curr_s_score > best_s_score:
                    best_s_score = curr_s_score
                    best_s_weight = [ensemble_weight[0],w,w2]
                if curr_h_score > best_h_score:
                    best_h_score = curr_h_score
                    best_h_weight = [ensemble_weight[0],w,w2]
        print(f'best soft score: {best_s_score}, best weight: {best_s_weight}')
        print(f'best hard score: {best_h_score}, best weight: {best_h_weight}')
        
        is_soft = True if best_s_score>=best_h_score else False
        ensemble_weight = best_s_weight if best_s_score>=best_h_score else best_h_weight


100%|██████████| 60/60 [13:31<00:00, 13.52s/it]

best soft score: 0.5652584219819898, best weight: [0.45, 0.45, 0.04]
best hard score: 0.567533815247799, best weight: [0.45, 0.46, 0.02]


In [8]:
    print(f'is_soft={is_soft}, is_valid={is_valid}, weight={ensemble_weight}')

is_soft=False, is_valid=True, weight=[0.45, 0.46, 0.02]


In [9]:
    ensemble_valid = pd.DataFrame()
    ensemble_valid['model_0_stat'] = m0_valid['label'].astype(int) - 1
    ensemble_valid['model_1_stat'] = m1_valid['label'].astype(int) - 1
    ensemble_valid['model_2_stat'] = m2_valid['label'].astype(int) - 1
    

In [10]:
    for stat in range(1, 4):
        ensemble_valid[f'{stat}_pred_prob'] = ensemble_weight[0] * m0_valid.loc[:, f'{stat}_pred_prob'] + \
                                              ensemble_weight[1] * m1_valid.loc[:, f'{stat}_pred_prob'] + \
                                              ensemble_weight[2] * m2_valid.loc[:, f'{stat}_pred_prob']
        m0_valid[f'{stat}_pred_label'] = 0
        m0_valid.loc[m0_valid[m0_valid.label == stat].index.tolist(), f'{stat}_pred_label'] = 1
        m1_valid[f'{stat}_pred_label'] = 0
        m1_valid.loc[m1_valid[m1_valid.label == stat].index.tolist(), f'{stat}_pred_label'] = 1
        m2_valid[f'{stat}_pred_label'] = 0
        m2_valid.loc[m2_valid[m2_valid.label == stat].index.tolist(), f'{stat}_pred_label'] = 1

        ensemble_valid[f'{stat}_pred_label'] = ensemble_weight[0] * m0_valid.loc[:, f'{stat}_pred_label'] + \
                                               ensemble_weight[1] * m1_valid.loc[:, f'{stat}_pred_label'] + \
                                               ensemble_weight[2] * m2_valid.loc[:, f'{stat}_pred_label']
        

In [11]:
    ensemble_valid['pred_stat_soft'] = ensemble_valid.loc[:, [f'{stat}_pred_prob' for stat in range(1, 4)]].idxmax(axis=1)
    ensemble_valid['pred_stat_soft'] = ensemble_valid['pred_stat_soft'].apply(lambda x: int(x.split('_')[0]) - 1).astype('int')
    ensemble_valid['pred_stat_hard'] = ensemble_valid.loc[:, [f'{stat}_pred_label' for stat in range(1, 4)]].idxmax(axis=1)
    ensemble_valid['pred_stat_hard'] = ensemble_valid['pred_stat_hard'].apply(lambda x: int(x.split('_')[0]) - 1).astype('int')
    

In [12]:
    if is_valid == True:
        
        print(f'soft ensemble valid F1 score: {f1_score_eval(ensemble_valid["pred_stat_soft"], valid["label"])}')
        print(f'hard ensemble valid F1 score: {f1_score_eval(ensemble_valid["pred_stat_hard"], valid["label"])}')
        print(f'model_0 valid F1 score: {f1_score_eval(ensemble_valid["model_0_stat"], valid["label"])}')
        print(f'model_1 valid F1 score: {f1_score_eval(ensemble_valid["model_1_stat"], valid["label"])}')
        print(f'model_2 valid F1 score: {f1_score_eval(ensemble_valid["model_2_stat"], valid["label"])}')
    

soft ensemble valid F1 score: 0.5642412209461809
hard ensemble valid F1 score: 0.567533815247799
model_0 valid F1 score: 0.562427175418077
model_1 valid F1 score: 0.5608858562717389
model_2 valid F1 score: 0.5509804811649321


In [13]:
    #is_soft=True
    #ensemble_weight=best_s_weight
    #print(f'is_soft={is_soft}, is_valid={is_valid}, weight={ensemble_weight}')

In [14]:
    is_valid = False # 如果下一步要生成文件，为False
    m0_valid = pd.read_csv(('valid' if is_valid is True else 'test')+m0)  # 模型1输出文件
    m1_valid = pd.read_csv(('valid' if is_valid is True else 'test')+m1)  # 模型2输出文件
    m2_valid = pd.read_csv(('valid' if is_valid is True else 'test')+m2)  # 模型2输出文件
    
    ensemble_valid = pd.DataFrame()
    ensemble_valid['model_0_stat'] = m0_valid['label'].astype(int) - 1
    ensemble_valid['model_1_stat'] = m1_valid['label'].astype(int) - 1
    ensemble_valid['model_2_stat'] = m2_valid['label'].astype(int) - 1
    
    for stat in range(1, 4):
        ensemble_valid[f'{stat}_pred_prob'] = ensemble_weight[0] * m0_valid.loc[:, f'{stat}_pred_prob'] + \
                                              ensemble_weight[1] * m1_valid.loc[:, f'{stat}_pred_prob'] + \
                                              ensemble_weight[2] * m2_valid.loc[:, f'{stat}_pred_prob']
        m0_valid[f'{stat}_pred_label'] = 0
        m0_valid.loc[m0_valid[m0_valid.label == stat].index.tolist(), f'{stat}_pred_label'] = 1
        m1_valid[f'{stat}_pred_label'] = 0
        m1_valid.loc[m1_valid[m1_valid.label == stat].index.tolist(), f'{stat}_pred_label'] = 1
        m2_valid[f'{stat}_pred_label'] = 0
        m2_valid.loc[m2_valid[m2_valid.label == stat].index.tolist(), f'{stat}_pred_label'] = 1

        ensemble_valid[f'{stat}_pred_label'] = ensemble_weight[0] * m0_valid.loc[:, f'{stat}_pred_label'] + \
                                               ensemble_weight[1] * m1_valid.loc[:, f'{stat}_pred_label'] + \
                                               ensemble_weight[2] * m2_valid.loc[:, f'{stat}_pred_label']
        
    ensemble_valid['pred_stat_soft'] = ensemble_valid.loc[:, [f'{stat}_pred_prob' for stat in range(1, 4)]].idxmax(axis=1)
    ensemble_valid['pred_stat_soft'] = ensemble_valid['pred_stat_soft'].apply(lambda x: int(x.split('_')[0]) - 1).astype('int')
    ensemble_valid['pred_stat_hard'] = ensemble_valid.loc[:, [f'{stat}_pred_label' for stat in range(1, 4)]].idxmax(axis=1)
    ensemble_valid['pred_stat_hard'] = ensemble_valid['pred_stat_hard'].apply(lambda x: int(x.split('_')[0]) - 1).astype('int')
    

In [15]:
    print(f'is_soft={is_soft}, is_valid={is_valid}, weight={ensemble_weight}')

is_soft=False, is_valid=False, weight=[0.45, 0.46, 0.02]


In [16]:
    if is_valid == False:
        print('using pred_stat_'+('soft' if is_soft is True else 'hard')+' for prediction')
        sub = pd.DataFrame()
        sub[['link', 'current_slice_id', 'future_slice_id']] = m2_valid.loc[:, ['link', 'current_slice_id', 'future_slice_id']]
        sub['label'] = ensemble_valid['pred_stat_'+('soft' if is_soft is True else 'hard')] + 1
        sub.to_csv('result_ensembling_new.csv', index=False, encoding='utf8')
    

using pred_stat_hard for prediction


In [ ]:
    # 以下cell无需执行

In [28]:
    # 四个模型权重测试
    is_valid = True

    print(f'is_soft={is_soft}, is_valid={is_valid}, weight={ensemble_weight}')
    m0_valid = pd.read_csv(('valid' if is_valid is True else 'test')+m0)  # 模型1输出文件
    m1_valid = pd.read_csv(('valid' if is_valid is True else 'test')+m1)  # 模型2输出文件
    m2_valid = pd.read_csv(('valid' if is_valid is True else 'test')+m2)  # 模型3输出文件
    m3 = '_prob_0.517.csv'
    m3_valid = pd.read_csv(('valid' if is_valid is True else 'test')+m3)  # 模型4输出文件
    ran = list(np.arange(0.0,0.6,0.01))
    best_s_score = 0
    best_h_score = 0
    for w in ran:
        
        ensemble_valid = pd.DataFrame()
        ensemble_valid['model_0_stat'] = m0_valid['label'].astype(int) - 1
        ensemble_valid['model_1_stat'] = m1_valid['label'].astype(int) - 1
        ensemble_valid['model_2_stat'] = m2_valid['label'].astype(int) - 1
        ensemble_valid['model_3_stat'] = m3_valid['label'].astype(int) - 1


        for stat in range(1, 4):
            ensemble_valid[f'{stat}_pred_prob'] = ensemble_weight[0] * m0_valid.loc[:, f'{stat}_pred_prob'] + \
                                                  ensemble_weight[1] * m1_valid.loc[:, f'{stat}_pred_prob'] + \
                                                  ensemble_weight[2] * m2_valid.loc[:, f'{stat}_pred_prob'] + \
                                                  w * m3_valid.loc[:, f'{stat}_pred_prob']
            m0_valid[f'{stat}_pred_label'] = 0
            m0_valid.loc[m0_valid[m0_valid.label == stat].index.tolist(), f'{stat}_pred_label'] = 1
            m1_valid[f'{stat}_pred_label'] = 0
            m1_valid.loc[m1_valid[m1_valid.label == stat].index.tolist(), f'{stat}_pred_label'] = 1
            m2_valid[f'{stat}_pred_label'] = 0
            m2_valid.loc[m2_valid[m2_valid.label == stat].index.tolist(), f'{stat}_pred_label'] = 1
            m3_valid[f'{stat}_pred_label'] = 0
            m3_valid.loc[m3_valid[m3_valid.label == stat].index.tolist(), f'{stat}_pred_label'] = 1

            ensemble_valid[f'{stat}_pred_label'] = ensemble_weight[0] * m0_valid.loc[:, f'{stat}_pred_label'] + \
                                           ensemble_weight[1] * m1_valid.loc[:, f'{stat}_pred_label'] + \
                                           ensemble_weight[2] * m2_valid.loc[:, f'{stat}_pred_label'] + \
                                           w * m3_valid.loc[:, f'{stat}_pred_label']

        ensemble_valid['pred_stat_soft'] = ensemble_valid.loc[:, [f'{stat}_pred_prob' for stat in range(1, 4)]].idxmax(axis=1)
        ensemble_valid['pred_stat_soft'] = ensemble_valid['pred_stat_soft'].apply(lambda x: int(x.split('_')[0]) - 1).astype('int')
        ensemble_valid['pred_stat_hard'] = ensemble_valid.loc[:, [f'{stat}_pred_label' for stat in range(1, 4)]].idxmax(axis=1)
        ensemble_valid['pred_stat_hard'] = ensemble_valid['pred_stat_hard'].apply(lambda x: int(x.split('_')[0]) - 1).astype('int')

        curr_s_score = f1_score_eval(ensemble_valid["pred_stat_soft"], valid["label"])
        curr_h_score = f1_score_eval(ensemble_valid["pred_stat_hard"], valid["label"])
        #print(f'weight: {w},{w2}, soft ensemble valid F1 score: {curr_score}')
        if curr_s_score > best_s_score:
            best_s_score = curr_s_score
            best_s_weight = [ensemble_weight[0],ensemble_weight[1],ensemble_weight[2],w]
        if curr_h_score > best_h_score:
            best_h_score = curr_h_score
            best_h_weight = [ensemble_weight[0],ensemble_weight[1],ensemble_weight[2],w]
    print(f'best soft score: {best_s_score}, best weight: {best_s_weight}')
    print(f'best hard score: {best_h_score}, best weight: {best_h_weight}')
    

is_soft=False, is_valid=True, weight=[0.45, 0.14, 0.27]
best soft score: 0.5653068934533243, best weight: [0.45, 0.14, 0.27, 0.0]
best hard score: 0.562427175418077, best weight: [0.45, 0.14, 0.27, 0.0]


In [28]:
    if is_valid == False:
        sub = m1_valid[['link', 'current_slice_id', 'future_slice_id']].deepcopy()
        sub.loc[:,'label'] = ensemble_valid['pred_stat_'+('soft' if is_soft is True else 'hard')] + 1
        sub.to_csv('result_ensembling_56196.csv', index=False, encoding='utf8')
    

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
